In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/creditcarddefault/credit-card-default.csv


In [2]:
credit = pd.read_csv('/kaggle/input/creditcarddefault/credit-card-default.csv')
credit.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,defaulted
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [3]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   ID         30000 non-null  int64
 1   LIMIT_BAL  30000 non-null  int64
 2   SEX        30000 non-null  int64
 3   EDUCATION  30000 non-null  int64
 4   MARRIAGE   30000 non-null  int64
 5   AGE        30000 non-null  int64
 6   PAY_0      30000 non-null  int64
 7   PAY_2      30000 non-null  int64
 8   PAY_3      30000 non-null  int64
 9   PAY_4      30000 non-null  int64
 10  PAY_5      30000 non-null  int64
 11  PAY_6      30000 non-null  int64
 12  BILL_AMT1  30000 non-null  int64
 13  BILL_AMT2  30000 non-null  int64
 14  BILL_AMT3  30000 non-null  int64
 15  BILL_AMT4  30000 non-null  int64
 16  BILL_AMT5  30000 non-null  int64
 17  BILL_AMT6  30000 non-null  int64
 18  PAY_AMT1   30000 non-null  int64
 19  PAY_AMT2   30000 non-null  int64
 20  PAY_AMT3   30000 non-null  int64
 21  PAY_AMT4   3

In [4]:
X = credit.drop('defaulted', axis=1)
Y = credit['defaulted']
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.3, random_state=101)

In [5]:
model = RandomForestClassifier()
model.fit(X_train, Y_train)

RandomForestClassifier()

In [6]:
predicted = model.predict(X_test)

In [7]:
print('Accuracy', round(accuracy_score(predicted, Y_test)*100))

Accuracy 82.0


In [8]:
n_folds = 5
parameters = {'max_depth':range(2,20,5)}

In [9]:
rfModel = RandomForestClassifier()
rfModel = GridSearchCV(rfModel, parameters, cv=n_folds, scoring='accuracy')

In [10]:
rfModel.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': range(2, 20, 5)}, scoring='accuracy')

In [11]:
scores = rfModel.cv_results_
pd.DataFrame(scores).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.046139,0.023548,0.032891,0.000755,2,{'max_depth': 2},0.801429,0.805952,0.801429,0.794762,0.798571,0.800429,0.003690,4
1,2.602237,0.014975,0.053449,0.004826,7,{'max_depth': 7},0.817381,0.820476,0.819048,0.815714,0.818571,0.818238,0.001605,1
2,3.900304,0.011916,0.070414,0.001159,12,{'max_depth': 12},0.811667,0.820000,0.820238,0.812857,0.818333,0.816619,0.003637,2
3,4.860931,0.019374,0.091470,0.000597,17,{'max_depth': 17},0.814048,0.815238,0.818333,0.811190,0.814048,0.814571,0.002305,3


In [12]:
# GridSearchCV to find optimal n_estimators
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


# specify number of folds for k-fold CV
n_folds = 5

# parameters to build the model on
parameters = {'n_estimators': range(100, 1500, 400)}

# instantiate the model (note we are specifying a max_depth)
rf = RandomForestClassifier(max_depth=4)


# fit tree on training data
rf = GridSearchCV(rf, parameters, 
                    cv=n_folds, 
                   scoring="accuracy")
rf.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(max_depth=4),
             param_grid={'n_estimators': range(100, 1500, 400)},
             scoring='accuracy')

In [13]:
# scores of GridSearch CV
scores = rf.cv_results_
pd.DataFrame(scores).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.687886,0.018590,0.041452,0.004002,100,{'n_estimators': 100},0.811667,0.812619,0.811190,0.805000,0.809762,0.810048,0.002687,1
1,8.286857,0.057533,0.186105,0.007361,500,{'n_estimators': 500},0.811429,0.814048,0.810238,0.806190,0.806905,0.809762,0.002908,2
2,14.903026,0.089655,0.348147,0.041183,900,{'n_estimators': 900},0.810714,0.814048,0.811190,0.804048,0.807857,0.809571,0.003389,4
3,21.795272,0.422718,0.482577,0.007098,1300,{'n_estimators': 1300},0.810952,0.813571,0.810952,0.805476,0.807381,0.809667,0.002875,3


In [14]:
# GridSearchCV to find optimal max_features
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


# specify number of folds for k-fold CV
n_folds = 5

# parameters to build the model on
parameters = {'max_features': [4, 8, 14, 20, 24]}

# instantiate the model
rf = RandomForestClassifier(max_depth=4)


# fit tree on training data
rf = GridSearchCV(rf, parameters, 
                    cv=n_folds, 
                   scoring="accuracy")
rf.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(max_depth=4),
             param_grid={'max_features': [4, 8, 14, 20, 24]},
             scoring='accuracy')

In [15]:
# scores of GridSearch CV
scores = rf.cv_results_
pd.DataFrame(scores).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_features,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.681766,0.013912,0.039299,0.000360,4,{'max_features': 4},0.809048,0.815238,0.810952,0.806905,0.807619,0.809952,0.002983,5
1,2.854023,0.016409,0.039236,0.000634,8,{'max_features': 8},0.817381,0.821190,0.821190,0.817857,0.817381,0.819000,0.001797,1
2,4.621352,0.027150,0.038901,0.000291,14,{'max_features': 14},0.817857,0.821429,0.819048,0.817381,0.818571,0.818857,0.001408,2
3,6.394748,0.026267,0.038946,0.000492,20,{'max_features': 20},0.817619,0.820952,0.819048,0.816905,0.817619,0.818429,0.001441,4
4,7.568579,0.024036,0.038675,0.000252,24,{'max_features': 24},0.817143,0.822381,0.819286,0.815476,0.818810,0.818619,0.002310,3


In [16]:
# GridSearchCV to find optimal min_samples_leaf
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


# specify number of folds for k-fold CV
n_folds = 5

# parameters to build the model on
parameters = {'min_samples_leaf': range(100, 400, 50)}

# instantiate the model
rf = RandomForestClassifier()


# fit tree on training data
rf = GridSearchCV(rf, parameters, 
                    cv=n_folds, 
                   scoring="accuracy")
rf.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'min_samples_leaf': range(100, 400, 50)},
             scoring='accuracy')

In [17]:
# scores of GridSearch CV
scores = rf.cv_results_
pd.DataFrame(scores).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,3.087231,0.031153,0.060276,0.000865,100,{'min_samples_leaf': 100},0.816667,0.816429,0.813571,0.813571,0.810476,0.814143,0.002267,1
1,2.778287,0.045042,0.056128,0.000209,150,{'min_samples_leaf': 150},0.809762,0.820238,0.810714,0.805000,0.807619,0.810667,0.005174,2
2,2.579761,0.023189,0.054499,0.001563,200,{'min_samples_leaf': 200},0.810952,0.814048,0.811429,0.805476,0.808810,0.810143,0.002867,3
3,2.436808,0.028102,0.061440,0.013069,250,{'min_samples_leaf': 250},0.811190,0.813810,0.810238,0.805000,0.808095,0.809667,0.002968,4
4,2.324059,0.036253,0.051267,0.001084,300,{'min_samples_leaf': 300},0.808810,0.809762,0.807619,0.805238,0.803333,0.806952,0.002359,5


In [18]:
# GridSearchCV to find optimal min_samples_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


# specify number of folds for k-fold CV
n_folds = 5

# parameters to build the model on
parameters = {'min_samples_split': range(200, 500, 50)}

# instantiate the model
rf = RandomForestClassifier()


# fit tree on training data
rf = GridSearchCV(rf, parameters, 
                    cv=n_folds, 
                   scoring="accuracy")
rf.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'min_samples_split': range(200, 500, 50)},
             scoring='accuracy')

In [19]:
# scores of GridSearch CV
scores = rf.cv_results_
pd.DataFrame(scores).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,4.077951,0.032916,0.066871,0.000558,200,{'min_samples_split': 200},0.817381,0.822857,0.821667,0.815000,0.818810,0.819143,0.002848,1
1,3.935510,0.053491,0.064090,0.000432,250,{'min_samples_split': 250},0.818571,0.821190,0.822143,0.815952,0.816667,0.818905,0.002431,2
2,3.800657,0.029979,0.062514,0.000907,300,{'min_samples_split': 300},0.818333,0.820476,0.820238,0.815238,0.816667,0.818190,0.002024,5
3,3.696625,0.025896,0.060976,0.000461,350,{'min_samples_split': 350},0.818810,0.820476,0.820238,0.816667,0.818095,0.818857,0.001408,3
4,4.939966,1.741297,0.086026,0.054103,400,{'min_samples_split': 400},0.817857,0.819524,0.820476,0.814762,0.816190,0.817762,0.002095,6


In [20]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [4,8,10],
    'min_samples_leaf': range(100, 400, 200),
    'min_samples_split': range(200, 500, 200),
    'n_estimators': [100,200, 300], 
    'max_features': [5, 10]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1,verbose = 1)

In [21]:
# Fit the grid search to the data
grid_search.fit(X_train, Y_train)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   55.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:  7.7min finished


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [4, 8, 10], 'max_features': [5, 10],
                         'min_samples_leaf': range(100, 400, 200),
                         'min_samples_split': range(200, 500, 200),
                         'n_estimators': [100, 200, 300]},
             verbose=1)

In [22]:
# printing the optimal accuracy score and hyperparameters
print('We can get accuracy of',grid_search.best_score_,'using',grid_search.best_params_)

We can get accuracy of 0.8183809523809523 using {'max_depth': 4, 'max_features': 10, 'min_samples_leaf': 100, 'min_samples_split': 200, 'n_estimators': 200}


In [23]:
# model with the best hyperparameters
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(bootstrap=True,
                             max_depth=10,
                             min_samples_leaf=100, 
                             min_samples_split=200,
                             max_features=10,
                             n_estimators=100)

In [24]:
# fit
rfc.fit(X_train,Y_train)

RandomForestClassifier(max_depth=10, max_features=10, min_samples_leaf=100,
                       min_samples_split=200)

In [25]:
# predict
predictions = rfc.predict(X_test)

In [26]:
# evaluation metrics
from sklearn.metrics import classification_report,confusion_matrix

In [27]:
print(classification_report(Y_test,predictions))

              precision    recall  f1-score   support

           0       0.84      0.96      0.90      7058
           1       0.70      0.36      0.47      1942

    accuracy                           0.83      9000
   macro avg       0.77      0.66      0.68      9000
weighted avg       0.81      0.83      0.81      9000



In [28]:
print(confusion_matrix(Y_test,predictions))

[[6757  301]
 [1249  693]]


In [29]:
((6752+689)/(6752+306+1253+689))

0.8267777777777777

# ** Final Model Accuracy : 82% **